In [1]:
# 나무위키 데이터셋 실험
from auto_preprocessing import word_preprocessing
from auto_preprocessing import autoencode
from auto_preprocessing import autocheck
import pickle
import os
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import os, pickle


data_dict = {}

# 모든 파일 읽기
for root, dirs, files in os.walk("../data/namu"):
    for file in files:
        with open(os.path.join(root, file), "r", encoding="utf-8") as f:
            data = f.read()
            data_dict[file] = data

In [2]:
# 단어 슬라이드 조절

slides = [ 1,3,5,7,9 ]
for slide in slides:
    result_dict = word_preprocessing(data_dict, slide=slide)
    with open(f"./result/preprocess/slide_{slide}.pkl", "wb") as f:
        pickle.dump(result_dict, f)

In [3]:
autoencode()

Encoding: 100%|███████████████████████████████████████████████████████| 6/6 [00:10<00:00,  1.70s/it]


In [4]:
autocheck()

Checking: 100%|███████████████████████████████████████████████████████| 6/6 [00:01<00:00,  5.15it/s]
